In [1]:
from loader import MoleculeDataset, SeqDataset,SeqMolDataset#########################
import torch
from torchvision.models import resnet18
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse

from torch_geometric.loader import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
import numpy as np

from model import GNN, GNN_graphpred
from sklearn.metrics import roc_auc_score

from splitters import scaffold_split,scaffold_split_1
import pandas as pd
import os
import shutil
from tensorboardX import SummaryWriter

criterion = nn.BCEWithLogitsLoss(reduction = "none")

In [2]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch implementation of pre-training of graph neural networks')
parser.add_argument('--device', type=int, default=0,
                        help='which gpu to use if any (default: 0)')#0000
parser.add_argument('--batch_size', type=int, default=32,
                        help='input batch size for training (default: 32)')
parser.add_argument('--epochs', type=int, default=100,
                        help='number of epochs to train (default: 100)')
parser.add_argument('--lr', type=float, default=0.001,
                        help='learning rate (default: 0.001)')
parser.add_argument('--lr_scale', type=float, default=1,
                        help='relative learning rate for the feature extraction layer (default: 1)')
parser.add_argument('--decay', type=float, default=0,
                        help='weight decay (default: 0)')
parser.add_argument('--num_layer', type=int, default=5,
                        help='number of GNN message passing layers (default: 5).')
parser.add_argument('--emb_dim', type=int, default=300,
                        help='embedding dimensions (default: 300)')
parser.add_argument('--dropout_ratio', type=float, default=0.5,
                        help='dropout ratio (default: 0.5)')
parser.add_argument('--graph_pooling', type=str, default="mean",
                        help='graph level pooling (sum, mean, max, set2set, attention)')
parser.add_argument('--JK', type=str, default="last",
                        help='how the node features across layers are combined. last, sum, max or concat')
parser.add_argument('--gnn_type', type=str, default="gin")
parser.add_argument('--dataset', type=str, default = 'affinity', help='root directory of dataset. For now, only classification.')
#parser.add_argument('--input_model_file', type=str, default = 'None', help='filename to read the model (if there is any)')
parser.add_argument('--input_model_file', type=str, default = 'Mole-BERT', help='filename to read the model (if there is any)')
parser.add_argument('--filename', type=str, default = '', help='output filename')
parser.add_argument('--seed', type=int, default=42, help = "Seed for splitting the dataset.")
parser.add_argument('--runseed', type=int, default=0, help = "Seed for minibatch selection, random initialization.")
parser.add_argument('--split', type = str, default="scaffold", help = "random or scaffold or random_scaffold")
parser.add_argument('--eval_train', type=int, default = 1, help='evaluating training or not')
parser.add_argument('--num_workers', type=int, default = 4, help='number of workers for dataset loading')
args = parser.parse_args(args=[])###############33

In [3]:
gnn_dataset = MoleculeDataset("./dataset/" + args.dataset, dataset=args.dataset)###########################转换成了分子图的格式

In [4]:
seq_dataset=SeqDataset('./dataset/affinity/processed/sequence.csv')
seq_dataset[2]

'QANLMRLKSDLFNRSPMYPGPTKDDPLTVTLGFTLQDIVKVDSSTNEVDLVYYEQQRWKLNSLMWDPNEYGNITDFRTSAADIWTPDITAYSSTRPVQVLSPQIAVVTHDGSVMFIPAQRLSFMCDPTGVDSEEGVTCAVKFGSWVYSGFEIDLKTDTDQVDLSSYYASSKYEILSATQTRQVQHYSCCPEPYIDVNLVVKFRER'

In [5]:
seq_gnn_dataset=SeqMolDataset(seq_dataset,gnn_dataset)
seq_dataset=seq_gnn_dataset.datasetA
mol_dataset=seq_gnn_dataset.datasetB
seq=seq_gnn_dataset.datasetA[3]
#print(seq)
data=seq_gnn_dataset.datasetB[3]
#print(data)
seq_mol_dataloader=DataLoader(seq_gnn_dataset)
seq_dataloader=DataLoader(seq_dataset)
mol_dataloader=DataLoader(mol_dataset)
class SeqMolDataLoader(DataLoader):
    def __init__(self, dataloaderA,dataloaderB):
        #super(SeqMolDataLoader, self).__init__()
        self.dataloaderA=dataloaderA
        self.dataloaderB=dataloaderB
        #self.data,self.slices=datasetB
    def __getitem__(self,idx):
        seqloader=dataloaderA[idx]
        '''
        data = Data()
        # data_masked = self.transform(data)
        # print('data', self.data.edge_index)
        # print('slices', self.slices)
        for key in self.data.keys:
            print('key:',key)
            item, slices = self.data[key], self.slices[key]
            print('item,slices:',item,slices)
            s = list(repeat(slice(None), item.dim()))
            s[data.__cat_dim__(key, item)] = slice(slices[idx],
                                                    slices[idx + 1])
            data[key] = item[s]
       '''
        molloader=dataloaderB[idx]
        return seqloader,molloader
    def __len__(self):
        return len(dataloaderA)
seq_mol_dataloader=SeqMolDataLoader(seq_dataloader,mol_dataloader)        

for i, (seq, mol) in enumerate(zip(seq_dataloader, mol_dataloader)):
    print(i, seq, mol)
'''

for i ,seq in enumerate(seq_mol_dataloader.dataloaderA):
    print(seq)

for i ,data in enumerate(seq_mol_dataloader.dataloaderB):
    print(data)
'''

key: x
item,slices: tensor([[5, 0],
        [5, 0],
        [5, 0],
        ...,
        [7, 0],
        [5, 0],
        [5, 0]]) tensor([   0,   10,   41,   66,   82,  118,  131,  157,  169,  193,  205,  241,
         262,  280,  309,  333,  349,  377,  392,  409,  430,  446,  470,  496,
         514,  544,  582,  599,  615,  666,  699,  720,  746,  780,  811,  842,
         873,  892,  927,  952,  968,  995, 1024, 1045, 1082, 1109, 1128, 1146,
        1170, 1207, 1225, 1250, 1287, 1303, 1321, 1339, 1389, 1408, 1446, 1466,
        1485, 1534, 1575, 1600, 1625, 1641, 1666, 1697, 1720, 1743, 1774, 1818,
        1838, 1851, 1898, 1922, 1955, 1994, 2017, 2031, 2071, 2107, 2128, 2150,
        2177, 2195, 2219, 2239, 2262, 2309, 2333, 2369, 2403, 2417, 2455, 2469,
        2502, 2532, 2542, 2570, 2593, 2606, 2628, 2657, 2702, 2733, 2757, 2805,
        2830, 2853, 2869, 2882, 2894, 2919, 2941, 2952, 2986, 3008, 3043, 3073,
        3100, 3122, 3140, 3162, 3175, 3193, 3213, 3238, 3251, 3260, 32

        [5, 0]]) tensor([   0,   10,   41,   66,   82,  118,  131,  157,  169,  193,  205,  241,
         262,  280,  309,  333,  349,  377,  392,  409,  430,  446,  470,  496,
         514,  544,  582,  599,  615,  666,  699,  720,  746,  780,  811,  842,
         873,  892,  927,  952,  968,  995, 1024, 1045, 1082, 1109, 1128, 1146,
        1170, 1207, 1225, 1250, 1287, 1303, 1321, 1339, 1389, 1408, 1446, 1466,
        1485, 1534, 1575, 1600, 1625, 1641, 1666, 1697, 1720, 1743, 1774, 1818,
        1838, 1851, 1898, 1922, 1955, 1994, 2017, 2031, 2071, 2107, 2128, 2150,
        2177, 2195, 2219, 2239, 2262, 2309, 2333, 2369, 2403, 2417, 2455, 2469,
        2502, 2532, 2542, 2570, 2593, 2606, 2628, 2657, 2702, 2733, 2757, 2805,
        2830, 2853, 2869, 2882, 2894, 2919, 2941, 2952, 2986, 3008, 3043, 3073,
        3100, 3122, 3140, 3162, 3175, 3193, 3213, 3238, 3251, 3260, 3293, 3317,
        3361, 3393, 3413, 3436, 3453, 3486, 3508, 3536, 3549, 3590, 3607, 3616,
        3632, 3667, 368

item,slices: tensor([ 2.3000,  4.1300,  7.4200,  6.8900, 11.1500,  5.1500,  7.8000,  5.6600,
         7.9600,  6.8200,  8.3500,  4.7600,  4.8500,  7.1200,  6.4400,  5.1600,
         4.6500,  3.5700,  5.2700,  5.5200,  3.2800,  9.0000,  8.0000,  6.6300,
         7.3000,  9.0000,  6.0000,  8.6600,  7.8000,  5.8200,  5.2200,  6.3200,
         7.1500,  5.7900,  8.4000,  8.0000,  6.4800,  9.5200,  7.0500,  7.8500,
         5.6000,  8.4000,  5.6200,  8.8500,  6.8300,  2.4000,  6.1000, 10.0000,
        11.3300,  4.5500,  8.3600,  6.7900,  4.0500,  6.5000,  5.3700,  7.8200,
         3.6200,  9.7200,  8.1000,  5.0000,  7.3200,  6.1100,  5.6400,  9.5200,
         3.1000,  4.8600,  6.8800,  5.7200,  5.9600,  9.2100,  9.1000,  4.0900,
         6.4600, 10.9200,  7.4000,  7.4700,  8.7200,  7.5200,  2.8500,  8.3000,
         6.4000,  9.5400,  8.5200,  7.3200,  7.8000,  4.1200,  5.5200,  8.7200,
         8.7400,  7.4000,  8.2500,  9.1000,  2.9200,  8.4800,  4.8900,  5.9600,
        10.7000,  5.1000,  

key: x
item,slices: tensor([[5, 0],
        [5, 0],
        [5, 0],
        ...,
        [7, 0],
        [5, 0],
        [5, 0]]) tensor([   0,   10,   41,   66,   82,  118,  131,  157,  169,  193,  205,  241,
         262,  280,  309,  333,  349,  377,  392,  409,  430,  446,  470,  496,
         514,  544,  582,  599,  615,  666,  699,  720,  746,  780,  811,  842,
         873,  892,  927,  952,  968,  995, 1024, 1045, 1082, 1109, 1128, 1146,
        1170, 1207, 1225, 1250, 1287, 1303, 1321, 1339, 1389, 1408, 1446, 1466,
        1485, 1534, 1575, 1600, 1625, 1641, 1666, 1697, 1720, 1743, 1774, 1818,
        1838, 1851, 1898, 1922, 1955, 1994, 2017, 2031, 2071, 2107, 2128, 2150,
        2177, 2195, 2219, 2239, 2262, 2309, 2333, 2369, 2403, 2417, 2455, 2469,
        2502, 2532, 2542, 2570, 2593, 2606, 2628, 2657, 2702, 2733, 2757, 2805,
        2830, 2853, 2869, 2882, 2894, 2919, 2941, 2952, 2986, 3008, 3043, 3073,
        3100, 3122, 3140, 3162, 3175, 3193, 3213, 3238, 3251, 3260, 32

        196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206])
key: edge_attr
item,slices: tensor([[3, 0],
        [3, 0],
        [3, 0],
        ...,
        [0, 0],
        [3, 0],
        [3, 0]]) tensor([    0,    20,    86,   148,   184,   262,   290,   346,   372,   426,
          450,   528,   576,   612,   672,   724,   758,   818,   850,   886,
          932,   968,  1022,  1080,  1120,  1186,  1270,  1308,  1342,  1452,
         1524,  1568,  1626,  1700,  1770,  1834,  1902,  1946,  2024,  2078,
         2112,  2168,  2230,  2278,  2362,  2422,  2462,  2502,  2556,  2638,
         2678,  2736,  2816,  2848,  2888,  2928,  3034,  3074,  3158,  3198,
         3240,  3346,  3436,  3496,  3552,  3584,  3640,  3706,  3756,  3808,
         3876,  3974,  4016,  4044,  4144,  4196,  4268,  4354,  4404,  4436,
         4526,  4606,  4654,  4702,  4760,  4796,  4848,  4892,  4944,  5044,
         5094,  5180,  5254,  5284,  5368,  5398,  5470,  5536,  5556,  5618,
         5670,  56

...

166 ['GTILIDLSPDDKEFQSVEEEMQSTVREHRDGGHAGGIFNRYNILKIQKVCNKKLWERYTHRRKEVSEENHNHANERMLFHGSPFVNAIIHKGFDERHAYIGGMFGAGIYFAENSSKSNQYVYGIGGGTGCPVHKDRSCYICHRQLLFCRVTLGKSFLQFS'] DataBatch(x=[21, 2], id=[1], edge_attr=[46, 2], y=[1], edge_index=[2, 46], batch=[21], ptr=[2])
key: x
item,slices: tensor([[5, 0],
        [5, 0],
        [5, 0],
        ...,
        [7, 0],
        [5, 0],
        [5, 0]]) tensor([   0,   10,   41,   66,   82,  118,  131,  157,  169,  193,  205,  241,
         262,  280,  309,  333,  349,  377,  392,  409,  430,  446,  470,  496,
         514,  544,  582,  599,  615,  666,  699,  720,  746,  780,  811,  842,
         873,  892,  927,  952,  968,  995, 1024, 1045, 1082, 1109, 1128, 1146,
        1170, 1207, 1225, 1250, 1287, 1303, 1321, 1339, 1389, 1408, 1446, 1466,
        1485, 1534, 1575, 1600, 1625, 1641, 1666, 1697, 1720, 1743, 1774, 1818,
        1838, 1851, 1898, 1922, 1955, 1994, 2017, 2031, 2071, 2107, 2128, 2150,
        2177, 2195, 2219, 2239, 2262, 2309, 2

        272, 274, 275, 276, 277, 278, 279, 280, 282, 284]) tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
     

        [3, 0]]) tensor([    0,    20,    86,   148,   184,   262,   290,   346,   372,   426,
          450,   528,   576,   612,   672,   724,   758,   818,   850,   886,
          932,   968,  1022,  1080,  1120,  1186,  1270,  1308,  1342,  1452,
         1524,  1568,  1626,  1700,  1770,  1834,  1902,  1946,  2024,  2078,
         2112,  2168,  2230,  2278,  2362,  2422,  2462,  2502,  2556,  2638,
         2678,  2736,  2816,  2848,  2888,  2928,  3034,  3074,  3158,  3198,
         3240,  3346,  3436,  3496,  3552,  3584,  3640,  3706,  3756,  3808,
         3876,  3974,  4016,  4044,  4144,  4196,  4268,  4354,  4404,  4436,
         4526,  4606,  4654,  4702,  4760,  4796,  4848,  4892,  4944,  5044,
         5094,  5180,  5254,  5284,  5368,  5398,  5470,  5536,  5556,  5618,
         5670,  5698,  5746,  5810,  5908,  5974,  6024,  6130,  6186,  6236,
         6272,  6300,  6324,  6380,  6428,  6450,  6530,  6580,  6658,  6726,
         6784,  6832,  6870,  6918,  6944,  698

key: y
item,slices: tensor([ 2.3000,  4.1300,  7.4200,  6.8900, 11.1500,  5.1500,  7.8000,  5.6600,
         7.9600,  6.8200,  8.3500,  4.7600,  4.8500,  7.1200,  6.4400,  5.1600,
         4.6500,  3.5700,  5.2700,  5.5200,  3.2800,  9.0000,  8.0000,  6.6300,
         7.3000,  9.0000,  6.0000,  8.6600,  7.8000,  5.8200,  5.2200,  6.3200,
         7.1500,  5.7900,  8.4000,  8.0000,  6.4800,  9.5200,  7.0500,  7.8500,
         5.6000,  8.4000,  5.6200,  8.8500,  6.8300,  2.4000,  6.1000, 10.0000,
        11.3300,  4.5500,  8.3600,  6.7900,  4.0500,  6.5000,  5.3700,  7.8200,
         3.6200,  9.7200,  8.1000,  5.0000,  7.3200,  6.1100,  5.6400,  9.5200,
         3.1000,  4.8600,  6.8800,  5.7200,  5.9600,  9.2100,  9.1000,  4.0900,
         6.4600, 10.9200,  7.4000,  7.4700,  8.7200,  7.5200,  2.8500,  8.3000,
         6.4000,  9.5400,  8.5200,  7.3200,  7.8000,  4.1200,  5.5200,  8.7200,
         8.7400,  7.4000,  8.2500,  9.1000,  2.9200,  8.4800,  4.8900,  5.9600,
        10.7000,  5.

'\n\nfor i ,seq in enumerate(seq_mol_dataloader.dataloaderA):\n    print(seq)\n\nfor i ,data in enumerate(seq_mol_dataloader.dataloaderB):\n    print(data)\n'

In [10]:
if args.split == "scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        
        #print('smiles_list:',smiles_list)
        mol_train_dataset, mol_valid_dataset, mol_test_dataset ,seq_train_dataset,seq_valid_dataset,seq_test_dataset= scaffold_split_1(seq_gnn_dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1)##########dataset
        print("scaffold")
elif args.split == "random":
        train_dataset, valid_dataset, test_dataset = random_split(dataset, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        print("random")
elif args.split == "random_scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        train_dataset, valid_dataset, test_dataset = random_scaffold_split(dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        print("random scaffold")
else:
        raise ValueError("Invalid split option.")

print('++++++++++', mol_train_dataset[0])


seq_mol_train_dataset=SeqMolDataset(seq_train_dataset,mol_train_dataset)
seq_mol_valid_dataset=SeqMolDataset(seq_valid_dataset,mol_valid_dataset)
seq_mol_test_dataset=SeqMolDataset(seq_train_dataset,mol_test_dataset)

seq_mol_train_loader = DataLoader(seq_mol_train_dataset, batch_size=args.batch_size, shuffle=True, num_workers = args.num_workers)
seq_mol_val_loader = DataLoader(seq_mol_valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)
seq_mol_test_loader = DataLoader(seq_mol_test_dataset, batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)






scaffold
key: edge_attr
item,slices: tensor([[3, 0],
        [3, 0],
        [3, 0],
        ...,
        [0, 0],
        [3, 0],
        [3, 0]]) tensor([    0,    20,    86,   148,   184,   262,   290,   346,   372,   426,
          450,   528,   576,   612,   672,   724,   758,   818,   850,   886,
          932,   968,  1022,  1080,  1120,  1186,  1270,  1308,  1342,  1452,
         1524,  1568,  1626,  1700,  1770,  1834,  1902,  1946,  2024,  2078,
         2112,  2168,  2230,  2278,  2362,  2422,  2462,  2502,  2556,  2638,
         2678,  2736,  2816,  2848,  2888,  2928,  3034,  3074,  3158,  3198,
         3240,  3346,  3436,  3496,  3552,  3584,  3640,  3706,  3756,  3808,
         3876,  3974,  4016,  4044,  4144,  4196,  4268,  4354,  4404,  4436,
         4526,  4606,  4654,  4702,  4760,  4796,  4848,  4892,  4944,  5044,
         5094,  5180,  5254,  5284,  5368,  5398,  5470,  5536,  5556,  5618,
         5670,  5698,  5746,  5810,  5908,  5974,  6024,  6130,  6186,  6

In [ ]:
def train(args, epoch, model, device, loader, optimizer):
    model.train()
    epoch_iter = tqdm(loader, desc="Iteration")
    for step, batch in enumerate(epoch_iter):
        batch = batch.to(device)
        print('batch:',batch.x,batch.edge_index,batch.edge_attr,batch.batch)
        pred, node_representation = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        y = batch.y.view(pred.shape).to(torch.float64)

        #Whether y is non-null or not.
        is_valid = y**2 > 0
        #Loss matrix
        loss_mat = criterion(pred.double(), (y+1)/2)
        #loss matrix after removing null target
        loss_mat = torch.where(is_valid, loss_mat, torch.zeros(loss_mat.shape).to(loss_mat.device).to(loss_mat.dtype))  
        optimizer.zero_grad()
        loss = torch.sum(loss_mat)/torch.sum(is_valid)
        loss.backward()
        optimizer.step()
        epoch_iter.set_description(f"Epoch: {epoch} tloss: {loss:.4f}")


In [11]:
a=['ab']
b=np.array(a)
print(b,type(b),b.shape)

['ab'] <class 'numpy.ndarray'> (1,)


In [14]:
import pandas as pd 
df=pd.read_csv('dataset/affinity/processed/smiles.csv',sep=',')
print(df)

                                  c1ccc(cc1)S(=O)(=O)N
0    Oc1cc(cc(C(=O)NCC=CC2OC(N3C=CC(=O)C=C3)C(O)C2O...
1      OC1N2c3ccccc3C34CC[N+]5CC6=CCOC(C1)C(C6CC35)C24
2                  C1=NC2=C(Sc3c(Cl)ccc(Cl)c32)C(=O)N1
3    C1SC(C(=O)Nc2c(OC)cc(Cl)cc2C(=O)Nc2ncc(Cl)cc2)...
4                            c1cc2C(=O)NC(=Nc2c(c1)C)N
..                                                 ...
200  OC1C(OC(COP(=O)(=O)OP(=O)(=O)=O)C1O)N1C=Nc2c(n...
201                       C(CC[N+]CCC[N+])C[N+]CCC[N+]
202                     OC1C(O)C2C(O)CC[N+]2CC1NC(=O)C
203  OC(C)C(C(=O)NCC(C)C)[N+]CC(NC(=O)c1cc(C(=O)NC(...
204        OC1C(OC(COP(=O)(=O)=O)C1O)N1C(=O)NC(=O)C=C1

[205 rows x 1 columns]


In [5]:
import numpy as np
a=np.array(['abc','ab'])
b=torch.tensor(a)
print(b)

TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [5]:
l=[1,2]
l.append('edd')
m=torch.tensor([0])
l.append(m)
print(l)

[1, 2, 'edd', tensor([0])]


In [29]:
#dataset test
import pandas as pd 
import torch
from torch.utils.data import Dataset,DataLoader
#from
df=pd.read_csv('dataset/affinity/processed/sequence.csv',sep=',',header=None)
#print(df)
class SeqDataset(Dataset):
    def __init__(self,seqPath):
        self.df_file=pd.read_csv(seqPath,sep=',',header=None)
        self.sequence_list=self.df_file.iloc[:,0]
        print(len(self.sequence_list))
    def __getitem__(self, idx):
        sequence=self.sequence_list[idx]
        print(idx)
        return sequence
    
    def __len__(self):
        return len(self.sequence_list)
    
seq_dataset=SeqDataset('dataset/affinity/processed/sequence.csv')
#print(seq_dataset)

#print(seq_dataset[3])
print(len(seq_dataset))
for i in range(len(seq_dataset)):
    print(i)
    print(seq_dataset[i])


206
206
0
0
HWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFCFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFNGEGEPEELMVDNWRPAQPLKNRQIKASFK
1
1
DTKEQRILRYVQQNAKPGDPQSVLEAIDTYCTQKEWAMNVGDAKGQIMDAVIREYSPSLVLELGAYCGYSAVRMARLLQPGARLLTMEINPDCAAITQQMLNFAGLQDKVTILNGASQDLIPQLKKKYDVDTLDMVFLDHWKDRYLPDTLLLEKCGLLRKGTVLLADNVIVPGTPDFLAYVRGSSSFECTHYSSYLEYMKVVDGLEKAIYQGP
2
2
QANLMRLKSDLFNRSPMYPGPTKDDPLTVTLGFTLQDIVKVDSSTNEVDLVYYEQQRWKLNSLMWDPNEYGNITDFRTSAADIWTPDITAYSSTRPVQVLSPQIAVVTHDGSVMFIPAQRLSFMCDPTGVDSEEGVTCAVKFGSWVYSGFEIDLKTDTDQVDLSSYYASSKYEILSATQTRQVQHYSCCPEPYIDVNLVVKFRER
3
3
PLESQYQVGPLLGSGGFGSVYSGIRVSDNLPVAIKHVEKDRISDWGELPNGTRVPMEVVLLKKVSSGFSGVIRLLDWFERPDSFVLILERPEPVQDLFDFITERGALQEELARSFFWQVLEAVRHCHNCGVLHRDIKDENILIDLNRGELKLIDFGSGALLKDTVYTDFDGTRVYSPPEWIRYHRYHGRSAAVWSLGILLYDMVCGDIPFEHDEEIIRGQVFFRQRVSSECQHLIRWCLALRPSDRPTFEEIQNHPWMQDVLLPQETAEIHLHS
4
4
IVGGQECKDGECPWQALLI

In [38]:
print(gnn_dataset[2])

key: x
item,slices: tensor([[5, 0],
        [5, 0],
        [5, 0],
        ...,
        [7, 0],
        [5, 0],
        [5, 0]]) tensor([   0,   10,   41,   66,   82,  118,  131,  157,  169,  193,  205,  241,
         262,  280,  309,  333,  349,  377,  392,  409,  430,  446,  470,  496,
         514,  544,  582,  599,  615,  666,  699,  720,  746,  780,  811,  842,
         873,  892,  927,  952,  968,  995, 1024, 1045, 1082, 1109, 1128, 1146,
        1170, 1207, 1225, 1250, 1287, 1303, 1321, 1339, 1389, 1408, 1446, 1466,
        1485, 1534, 1575, 1600, 1625, 1641, 1666, 1697, 1720, 1743, 1774, 1818,
        1838, 1851, 1898, 1922, 1955, 1994, 2017, 2031, 2071, 2107, 2128, 2150,
        2177, 2195, 2219, 2239, 2262, 2309, 2333, 2369, 2403, 2417, 2455, 2469,
        2502, 2532, 2542, 2570, 2593, 2606, 2628, 2657, 2702, 2733, 2757, 2805,
        2830, 2853, 2869, 2882, 2894, 2919, 2941, 2952, 2986, 3008, 3043, 3073,
        3100, 3122, 3140, 3162, 3175, 3193, 3213, 3238, 3251, 3260, 32

NameError: name 'sequence' is not defined